In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2/config.json
/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2/merges.txt
/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2/vocab.json
/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2/tokenizer_config.json
/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2/model.safetensors
/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2/special_tokens_map.json
/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2/generation_config.json
/kaggle/input/meddialogue/test_data.json
/kaggle/input/meddialogue/english-dev.json
/kaggle/input/meddialogue/healthcaremagic_dialogue_3.txt
/kaggle/input/meddialogue/train_data.json
/kaggle/input/meddialogue/healthcaremagic_dialogue_4.txt
/kaggle/input/meddialogue/healthcaremagic_dialogue_2.txt
/kaggle/input/meddialogue/healthcaremagic_dialogue_1.txt
/kaggle/input/meddialogue/en

In [1]:
!pip install -q torch transformers datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

In [2]:
import os
import json

# Importing MedDialog dataset 
dataset_path = '/kaggle/input/meddialogue'

# Check dataset is uploaded or not
print("Files in dataset folder:")
print(os.listdir(dataset_path))

# Loading train, validation, and test sets
with open(f"{dataset_path}/english-train.json", "r") as f:
    train_data = json.load(f)
with open(f"{dataset_path}/english-dev.json", "r") as f:
    val_data = json.load(f)
with open(f"{dataset_path}/english-test.json", "r") as f:
    test_data = json.load(f)


Files in dataset folder:
['test_data.json', 'english-dev.json', 'healthcaremagic_dialogue_3.txt', 'train_data.json', 'healthcaremagic_dialogue_4.txt', 'healthcaremagic_dialogue_2.txt', 'healthcaremagic_dialogue_1.txt', 'english-train.json', 'english-test.json', 'icliniq_dialogue.txt', 'validate_data.json']


In [3]:
from datasets import Dataset

#Extract question - answers pairs from the3 datasets
def format_dialogue_data(data):
    formatted = []
    for item in data:
        utterances = item.get("utterances", [])
        if len(utterances) >= 2 and utterances[0].lower().startswith("patient") and utterances[1].lower().startswith("doctor"):
            question = utterances[0].replace("patient:", "").strip()
            answer = utterances[1].replace("doctor:", "").strip()
            text = f"Patient: {question}\nDoctor: {answer} [Disclaimer: This is not medical advice.]"
            formatted.append({"text": text})
    return formatted

train_samples = format_dialogue_data(train_data)
val_samples = format_dialogue_data(val_data)
test_samples = format_dialogue_data(test_data)

train_dataset = Dataset.from_list(train_samples)
val_dataset = Dataset.from_list(val_samples)
test_dataset = Dataset.from_list(test_samples)

In [4]:
# Import classes from HuggingFace Transformers and PyTorch
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# using gpt-2 medium
model_name = "gpt2-medium"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-06-06 10:52:54.472860: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749207174.730142      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749207174.804248      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [5]:
# Tokenize function for all samples
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/482 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [6]:
from torch.utils.data import DataLoader

tokenized_train.set_format("torch", columns=["input_ids", "attention_mask"])
train_loader = DataLoader(tokenized_train, batch_size=2, shuffle=True)


In [7]:
#Now train the model 
from torch.optim import AdamW
from transformers import get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 2
num_training_steps = len(train_loader) * num_epochs

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


In [10]:
from tqdm import tqdm

model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    loop = tqdm(train_loader)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["input_ids"]
        )
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    # ✅ Save after each epoch
    checkpoint_path = f"mediguide_checkpoint_epoch{epoch+1}"
    model.save_pretrained(checkpoint_path)
    tokenizer.save_pretrained(checkpoint_path)


Epoch 1


Epoch 1: 100%|██████████| 241/241 [1:09:43<00:00, 17.36s/it, loss=1]    


Epoch 2


Epoch 2: 100%|██████████| 241/241 [1:09:15<00:00, 17.24s/it, loss=0.739]


In [11]:
#Save model to avoid run again the two epochs
model.save_pretrained("/kaggle/working/mediguide_checkpoint_epoch2")
tokenizer.save_pretrained("/kaggle/working/mediguide_checkpoint_epoch2")

!zip -r /kaggle/working/mediguide_checkpoint_epoch2.zip /kaggle/working/mediguide_checkpoint_epoch2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/mediguide_checkpoint_epoch2/ (stored 0%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/vocab.json (deflated 59%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/tokenizer.json (deflated 82%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/model.safetensors (deflated 7%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/tokenizer_config.json (deflated 54%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/merges.txt (deflated 53%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/config.json (deflated 52%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/special_tokens_map.json (deflated 60%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/generation_config.json (deflated 24%)


In [13]:
from IPython.display import FileLink
FileLink(r'mediguide_checkpoint_epoch2.zip')

/kaggle/working/mediguide_checkpoint_epoch2.zip

In [8]:
import os

path = "/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2"
print(os.listdir(path))


['config.json', 'merges.txt', 'vocab.json', 'tokenizer_config.json', 'model.safetensors', 'special_tokens_map.json', 'generation_config.json']


In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define local model path
model_path = "/kaggle/input/mediguide-checkpoint-chatbot/mediguide_checkpoint_epoch2"

# Load model and tokenizer from local files
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    local_files_only=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    local_files_only=True
)

tokenizer.pad_token = tokenizer.eos_token


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [9]:
import json
from datasets import Dataset

# Load validation data
with open("/kaggle/input/meddialogue/english-dev.json", "r") as f:
    val_data = json.load(f)

# Format it into the proper prompt/response format
def format_dialogue_data(data):
    formatted = []
    for item in data:
        utterances = item.get("utterances", [])
        if len(utterances) >= 2 and utterances[0].lower().startswith("patient") and utterances[1].lower().startswith("doctor"):
            question = utterances[0].replace("patient:", "").strip()
            answer = utterances[1].replace("doctor:", "").strip()
            text = f"Patient: {question}\nDoctor:"
            formatted.append({"prompt": text, "reference": answer})
    return formatted

eval_data = format_dialogue_data(val_data)


In [27]:
eval_data = eval_data[:100]  # Calculate Rouge score for 100 samples


In [10]:
model.eval()

generated_texts = []
reference_texts = []

for sample in eval_data:
    inputs = tokenizer(sample["prompt"], return_tensors="pt", truncation=True).to(device)

    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50, 
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if "Doctor:" in generated:
        generated_answer = generated.split("Doctor:")[-1].strip()
    else:
        generated_answer = generated.strip()

    generated_texts.append(generated_answer)
    reference_texts.append(sample["reference"])


In [11]:
from evaluate import load

rouge = load("rouge")

results = rouge.compute(predictions=generated_texts, references=reference_texts)


print("ROUGE Score is:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


ROUGE Score is:
rouge1: 0.0970
rouge2: 0.0092
rougeL: 0.0737
rougeLsum: 0.0757


In [12]:
import torch
import math

model.eval()
total_loss = 0
num_batches = 0

for sample in eval_data[:100]: 
    inputs = tokenizer(
        sample["prompt"] + sample["reference"],
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding="max_length"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        total_loss += loss.item()
        num_batches += 1

avg_loss = total_loss / num_batches
perplexity = math.exp(avg_loss)

print(f"Avg. Loss: {avg_loss:.4f}")
print(f"Perplexity : {perplexity:.2f}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Avg. Loss: 8.8642
Perplexity : 7074.03


In [13]:
from transformers import pipeline

# Create text generation pipeline using your model and tokenizer
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Sample prompt
prompt = "i have cold cough"

# Generate response
response = pipe(prompt, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

print("Model Response:\n", response[0]["generated_text"])


Device set to use cpu


Model Response:
 i have cold coughs, have vomiting and diarrhea, or are pregnant. Although this condition is not serious, anemia may prevent an entire pregnancy from being successfully delivered.

Tardive dyskinesia is a disorder associated with excessive breathing followed by loss of awareness, resulting in a severe, chronic inability to speak. The syndrome's onset generally occurs during childhood or adolescence. The patient has the most difficulty with swallowing, with vomiting being much more frequent. A complete absence of speech may continue for months, often
